In [4]:
import random
import datetime
import csv

In [17]:
class Student:
    """Student class"""
    
    def __init__(self, name, section, motivation, length):
        self.name = name
        self.section = section
        self.motivation = motivation
        self.successes = [0] * length
        self.section_start = datetime.datetime.strptime(section["start"],"%Y-%m-%d-%H")
    
    def attempt(self, week, gradebook):
        due = self.section_start + datetime.timedelta(weeks=week)
        effort = random.randrange(10) + self.motivation
        if (effort > 10):
            # due date - 3 days +- 12hourrange
            attempt_date = pick_time(due, -72, 12)
        elif (effort > 5):
            # due date - 12hourrange
            attempt_date = pick_time(due, -12, 12)
        elif (effort > 2):
            # due date + 6hourrange
            attempt_date = pick_time(due, -3, 3)
        else:
            # due date + 6days +- 24 hourrange
            attempt_date = pick_time(due, 72, 72)
        
        # make attempt if there's not previous sucesses, maybe make attempt if there are
        if (random.random()<(1 - 0.9*self.successes[week])):
            # check if attempt is right (70% chance each time)
            if (random.random() < 0.7):
                self.successes[week] = 1
                correct = True
            else:
                correct = False
                
            gradebook.append(
                (self.name,
                 self.section.get("label"),
                 attempt_date.strftime("%Y-%m-%d-%H"),
                 correct,
                 week+1)
            )
            
            self.attempt(week, gradebook)

            

class Course:
    """Course class"""

    def __init__(self, length, sections = 'deadlines.csv'):
        self.length = length
        self.students = []
        self.submissions = None
        
        if type(sections) == str:
            with open('deadlines.csv') as deadline_file:
                self.sections = list(csv.DictReader(deadline_file))
        else:
            self.sections=sections
            

        # Create students
        for section in self.sections:
            for i in range(int(section.get("size"))):
                name = str(random.randrange(1000000,9999999))
                motivation = random.randint(1,3)
                self.students.append(
                    Student(name, section, motivation, self.length)
                )

    def run(self):
        "simulates running a semester of the course"
        submissions = []
        for student in self.students:
            for week in range(0,self.length):
                student.attempt(week = week, gradebook = submissions)

        self.submissions = submissions
    
    def write_files(self):
        fieldnames = ['UID', 'Section','Submission date', 'Correct', 'week']
        for w in range(1,self.length+1):
            file_name = 'submissions_week%i' % w
            with open(file_name, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(fieldnames)
                # write file for each week
                for r in range(len(self.submissions)):
                    # check if row was for week w (current loop)
                    if self.submissions[r][-1] == w:
                        writer.writerow(self.submissions[r])

def pick_time(target, hours_offset, hours_spread):
        dhours = hours_offset + random.uniform(-hours_spread,hours_spread)
        delta = datetime.timedelta(hours=dhours)
        return target + delta

## Intended Flow 


```python
course = Course(sections, 16)
submissions = course.run()

gradebook = grade(submissions)
```

### `sections`
`sections` should be a `list` of `dict`s that are all of the form:
```python
AD1 = {
    "label":"AD1",
    "size": 12,
    "start": "2019-01-14-12"
}
```

In [18]:
AD1 = {
    "label":"AD1",
    "size": 13,
    "start": "2019-01-14-12"
}

AD2 = {
    "label":"AD2",
    "size": 14,
    "start": "2019-01-14-13"
}

AD3 = {
    "label":"AD3",
    "size": 11,
    "start": "2019-01-14-14"
}

AD4 = {
    "label":"AD4",
    "size": 12,
    "start": "2019-01-14-15"
}

AD5 = {
    "label":"AD5",
    "size": 23,
    "start": "2019-01-15-08"
}

AD6 = {
    "label":"AD6",
    "size": 17,
    "start": "2019-01-15-09"
}

AD7 = {
    "label":"AD7",
    "size": 16,
    "start": "2019-01-15-12"
}

ADA = {
    "label":"ADA",
    "size": 16,
    "start": "2019-01-15-13"
}

test_course = Course(length=3)

[{'section': 'AD1', 'start': '2019-01-14-12', 'size': '15'}, {'section': 'AD2', 'start': '2019-01-14-13', 'size': '12'}, {'section': 'AD3', 'start': '2019-01-14-14', 'size': '23'}, {'section': 'AD4', 'start': '2019-01-14-15', 'size': '17'}, {'section': 'AD5', 'start': '2019-01-15-08', 'size': '20'}, {'section': 'AD7', 'start': '2019-01-15-09', 'size': '16'}, {'section': 'AD6', 'start': '2019-01-15-10', 'size': '21'}, {'section': 'ADA', 'start': '2019-01-15-11', 'size': '16'}]


In [19]:
random.seed(0)
test_course.run()

In [20]:
len(test_course.submissions)

659

In [ ]:
test_course.write_files()